In [ ]:
%pip install -r ../requirements.txt

In [ ]:
dbutils.library.restartPython()

In [ ]:
from dbx.pixels.dicom import StowProcessor

In [ ]:
# Parameters passed by job_parameters.
catalog_table = dbutils.widgets.get("catalog_table")
volume = dbutils.widgets.get("volume")

assert catalog_table, "catalog_table is required"
assert volume, "volume is required"

_parts = catalog_table.split(".")
assert len(_parts) == 3, "catalog_table must be catalog.schema.table"
stow_table = f"{_parts[0]}.{_parts[1]}.stow_operations"

volume_path = f"/Volumes/{volume.replace('.', '/')}"
checkpoint = f"{volume_path}/_checkpoints/stow_meta_extract"

print(f"STOW tracking table : {stow_table}")
print(f"Catalog table       : {catalog_table}")
print(f"Volume              : {volume}")
print(f"Checkpoint          : {checkpoint}")


In [ ]:
# Phase 2: Read completed entries (from Phase 1) → DicomMetaExtractor → save to catalog.
# This task runs in the background after Phase 1 returns paths to the client.

processor = StowProcessor(spark=spark)

processor.extract_metadata(
    source_table=stow_table,
    catalog_table=catalog_table,
    volume=volume,
    checkpoint_location=checkpoint,
)
